In [ ]:
from pathlib import Path

import httpx
import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

In [ ]:
import datetime

In [ ]:
xls = pd.ExcelFile("../../data/stories.xlsx", engine="openpyxl")
df = pd.read_excel(xls, "Articles", dtype={"Date published": datetime.datetime})

In [ ]:
get_id = lambda x: Path(x).name

In [ ]:
df["id"] = df["URL"].apply(get_id)

In [ ]:
def get_slices(id):
    response = httpx.get(
        "https://wellcomecollection.cdn.prismic.io/api/v2/documents/search",
        params={"ref": "YZZ4mREAACcAWBme", "q": f'[[at(document.id, "{id}")]]'},
    ).json()
    slices = response["results"][0]["data"]["body"]
    return slices


def get_fulltext(id):
    slices = get_slices(id)
    paragraphs = [
        paragraph["text"]
        for slice in slices
        if slice["slice_type"] == "text"
        for paragraph in slice["primary"]["text"]
    ]
    return "\n".join(paragraphs)


def get_standfirst(id):
    slices = get_slices(id)
    paragraphs = [
        paragraph["text"]
        for slice in slices
        if slice["slice_type"] == "standfirst"
        for paragraph in slice["primary"]["text"]
    ]
    return "\n".join(paragraphs)

In [ ]:
df["fulltext"] = df["id"].progress_apply(get_fulltext)
df["standfirst"] = df["id"].progress_apply(get_standfirst)

In [ ]:
df.to_json("../../data/stories.json", orient="records")